In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from time import time
import math

In [ ]:
class Shared:
    def __init__(self, A, b, learning_rate):
        ## input matrix's shape
        self.length = A.shape[0]
        self.width = A.shape[1]
        
        ## in CPU
        self.A = np.float32(A)
        self.b = np.float32(b)
        self.theta = np.float32(np.zeros(self.width))
        self.out = np.float32(np.zeros(self.length))
        self.grad = np.float32(np.zeros(self.width))

        ## in GPU
        self.GPU_A = gpuarray.to_gpu(self.A.reshape(self.length*self.width))
        self.GPU_b = gpuarray.to_gpu(self.b)
        self.GPU_theta = gpuarray.to_gpu(self.theta)
        self.GPU_out = gpuarray.to_gpu(self.out)
        self.GPU_grad = gpuarray.to_gpu(self.grad)

        ## for initialize out vector and grad vector
        self.init_out = gpuarray.empty_like(self.GPU_out)
        self.init_grad = gpuarray.empty_like(self.GPU_grad)

        ## TPB: thread_per_block, BPG: block_per_grid        
        self.TPB, self.BPG = self.optimal_block_size(self.length)

        ## learning_rate
        self.learning_rate = learning_rate

    def optimal_block_size(self, n):
        
        thread_per_block = int(math.sqrt(n / 2))

        block_per_grid = int(n / thread_per_block) + 1


        return thread_per_block, block_per_grid

In [ ]:
class GetGradient:
    def __init__(self, shared):
       self.shared = shared
       
       self.kernel_function()

    def run(self):
        ## get out = np.dot(A, theta)
        self.first(self.shared.GPU_out,
                   self.shared.GPU_A,
                   self.shared.GPU_theta,
                   np.int32(self.shared.length),
                   np.int32(self.shared.width),
                   block=(self.shared.TPB,1,1),
                   grid=(self.shared.BPG,1,1))

        ## get out -= b
        self.second(self.shared.GPU_out,
                    self.shared.GPU_b,
                    np.int32(self.shared.length),
                    block=(self.shared.TPB,1,1),
                    grid=(self.shared.BPG,1,1))

        ## get grad = np.dot(A.T, out)
        self.third(self.shared.GPU_grad,
                   self.shared.GPU_A,
                   self.shared.GPU_out,
                   np.int32(self.shared.TPB),
                   np.int32(self.shared.BPG),
                   np.int32(self.shared.length),
                   np.int32(self.shared.width),
                   block=(self.shared.BPG,1,1),
                   grid=(self.shared.width,1,1))
                   
                   

    def kernel_function(self):
        ## block=(thread_per_block,1,1), grid=(block_per_grid,1,1)
        first_ker_function = \
        """
        #define x (threadIdx.x + blockIdx.x * blockDim.x)

        __global__ void first(float* out, float* A, float* theta, int length, int width) {
            
            if (x < length) {
                for (int j = 0; j < width; j++) {
                    int index1 = x * width + j;

                    out[x] += A[index1] * theta[j];
                    }
            }
        }
        """
        first_ker = SourceModule(first_ker_function)



        ## block=(thread_per_block,1,1), grid=(block_per_grid,1,1)
        second_ker_function = \
        """
        #define x (threadIdx.x + blockIdx.x * blockDim.x)

        __global__ void second(float* out, float* b, int length) {

            if (x < length) {
                out[x] = out[x] - b[x];
            }
        }
        """
        second_ker = SourceModule(second_ker_function)



        ## block=(block_per_grid,1,1), grid=(width,1,1)
        third_ker_function = \
        """
        #define tx (threadIdx.x)
        #define bx (blockIdx.x)

        __global__ void third(float* grad, float* A, float* out, int thread_per_block, int block_per_grid, int length, int width) {

            __shared__ float grad_jerk[1000];

            grad_jerk[tx] = 0;

            __syncthreads();
            
            for (int i = 0; i < thread_per_block; i++) {
                int index1 = tx * thread_per_block + i;
                int index2 = index1 * width + bx;
                
                grad_jerk[tx] += A[index2] * out[index1];
            }

            __syncthreads();

            if (tx == 0) {
                for (int i = 0; i < block_per_grid; i++) {
                    grad[bx] += grad_jerk[i];
                }
            }
            else {
                grad_jerk[1000-tx] = 0;
            }

            __syncthreads();
        }
        """
        third_ker = SourceModule(third_ker_function)

        self.first = first_ker.get_function("first")
        self.second = second_ker.get_function("second")
        self.third = third_ker.get_function("third")

In [ ]:
class GradientMethod:
    def __init__(self, shared):
        self.shared = shared

        self.kernel_function()

    def run(self):

        self.gradient_method(self.shared.GPU_theta,
                             self.shared.GPU_grad,
                             np.float32(self.shared.learning_rate),
                             block=(self.shared.width,1,1),
                             grid=(1,1,1))

    def kernel_function(self):

        ## block=(width,1,1), grid=(1,1,1)
        gradient_method_ker = SourceModule(
            """
            #define x (threadIdx.x)

            __global__ void gradient_method (float* theta, float* grad, float learning_rate) {
                theta[x] -= learning_rate * grad[x];

                __syncthreads();
            }
            """
        )

        self.gradient_method = gradient_method_ker.get_function("gradient_method")

In [ ]:
class MomentumMethod:
    def __init__(self, shared):
        self.shared = shared

        self.kernel_function()

    def run(self):
        
        self.momentum_method(self.shared.GPU_theta,
                             self.shared.GPU_grad,
                             self.shared.GPU_momentum,
                             np.float32(self.shared.s),
                             np.float32(self.shared.beta),
                             block=(self.shared.width,1,1),
                             grid=(1,1,1))

    def kernel_function(self):

        ## block=(width,1,1), grid=(1,1,1)
        momentum_method_ker = SourceModule(
            """
            #define x (threadIdx.x)

            __global__ void momentum_method (float* n_theta, float* theta, float* grad, float* n_momentum, float* momentum, float s, float beta) {
                n_theta[x] = theta[x] - s * momentum[x];

                n_momentum[x] = grad[x] + beta * momentum[x];

                __syncthreads();
            }
            """
        )
        
        self.momentum_method = momentum_method_ker.get_function("momentum_method")

In [ ]:
A = np.random.randn(1000,20)
b = np.random.randn(1000)
learning_rate = 2e-4

shared = Shared(A, b, learning_rate)

get_gradient = GetGradient(shared)

get_gradient.run()

In [ ]:
grad_cpu = np.dot(A.T, (np.dot(A, np.zeros(A.shape[1])) - b))
np.linalg.norm(shared.GPU_grad.get() - grad_cpu)

2.2414368471636293e-05

In [ ]:
gradient_method = GradientMethod(shared)

gradient_method.run()

In [ ]:
np.linalg.norm(shared.GPU_theta.get() - (np.zeros(A.shape[1]) - learning_rate * grad_cpu))

4.5706114221004795e-09